# Python State Machine
## Eine selbstgeschriebene SM

In [13]:
class StateMachine:
    def __init__(self):
        self.state = None  # Der aktuelle Zustand des Automaten
        self.transitions = {}  # Eine Tabelle der Übergänge zwischen Zuständen

    def add_state(self, state_name, is_initial=False):
        """Fügt einen neuen Zustand hinzu. Falls is_initial True ist, wird dieser Zustand als Anfangszustand festgelegt."""
        self.transitions[state_name] = {}
        if is_initial:
            self.state = state_name

    def add_transition(self, from_state, to_state, event):
        """Definiert einen Übergang von einem Zustand zu einem anderen bei einem bestimmten Ereignis."""
        if from_state in self.transitions:
            self.transitions[from_state][event] = to_state
        else:
            raise ValueError(f"Zustand '{from_state}' nicht definiert")

    def on_event(self, event):
        """Reagiert auf ein Ereignis und verändert den Zustand des Automaten entsprechend."""
        if self.state in self.transitions and event in self.transitions[self.state]:
            self.state = self.transitions[self.state][event]
        else:
            raise ValueError(f"Kein Übergang definiert für Zustand '{self.state}' bei Ereignis '{event}'")

    def get_state(self):
        """Gibt den aktuellen Zustand des Automaten zurück."""
        return self.state

# Beispiel für die Nutzung des Zustandsautomaten
fsm = StateMachine()
fsm.add_state("wait", is_initial=True)
fsm.add_state("wait_for_value")
fsm.add_state("wait_for_triggerId")
fsm.add_state("write_to_db")

fsm.add_transition("wait", "wait_for_value", "new_triggerId_event")
fsm.add_transition("wait", "wait_for_triggerId", "new_value_event")
fsm.add_transition("wait_for_triggerId", "write_to_db" , "new_triggerId_event")
fsm.add_transition("wait_for_value", "write_to_db" , "new_value_event")
fsm.add_transition("write_to_db" ,"wait", "db_ok")



print(fsm.get_state())  # Ausgabe: initial
fsm.on_event("new_triggerId_event")
print(fsm.get_state())  # Ausgabe: state1
fsm.on_event("new_value_event")
print(fsm.get_state())  # Ausgabe: state2
fsm.on_event("db_ok")
print(fsm.get_state())  # Ausgabe: state1

wait
wait_for_value
write_to_db
wait


# Eine StateMachine (SM) mit Transitions Library
https://github.com/pytransitions/transitions?tab=readme-ov-file#table-of-contents


In [26]:
from transitions import Machine, State

# Definition von Zuständen mit on_enter und on_exit Aktionen
class IdleState(State):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.on_enter.append(self.start_idle_task)
        self.on_exit.append(self.stop_idle_task)

    def start_idle_task(self):
        print("Entering idle state and starting idle task.")

    def stop_idle_task(self):
        print("Exiting idle state and stopping idle task.")

    def do(self):
        print("Idle...")

class WorkingState(State):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.on_enter.append(self.start_working_task)
        self.on_exit.append(self.stop_working_task)

    def start_working_task(self):
        print("Entering working state and starting working task.")

    def stop_working_task(self):
        print("Exiting working state and stopping working task.")

    def do(self):
        print("Working...")

# Definition der Aktionen und zentralen Aufgaben
class Worker(object):
    def __init__(self):
        self.states = {
            'idle': IdleState(name='idle'),
            'working': WorkingState(name='working')
        }
        self.machine = Machine(model=self, states=list(self.states.values()), initial='idle')

    def do(self):
        current_state = self.machine.get_state(self.state)
        if hasattr(current_state, 'do'):
            current_state.do()

# Maschine mit Zuständen initialisieren
worker = Worker()

# Übergänge hinzufügen
worker.machine.add_transition('start_working', 'idle', 'working')
worker.machine.add_transition('stop_working', 'working', 'idle')

# Zustand wechseln und zentrale Aufgaben ausführen
worker.do()             # Ausgabe: Idle...
worker.start_working()  # Ausgabe: Entering working state and starting working task.
worker.do()             # Ausgabe: Working...
worker.stop_working()   # Ausgabe: Exiting working state and stopping working task.
worker.do()             # Ausgabe: Idle...

Idle...
Exiting idle state and stopping idle task.
Entering working state and starting working task.
Working...
Exiting working state and stopping working task.
Entering idle state and starting idle task.
Idle...


## Asynchrone State Machine mit pytransitions und Asyncio Extension
Beachte aus [Frequently asked questions - pytransitions](https://github.com/pytransitions/transitions/blob/master/examples/Frequently%20asked%20questions.ipynb): I have several inter-dependent machines/models and experience deadlocks.
#### Übergabe eigener Parameter an die Transitions CallBacks
>You can pass any number of arguments you like to the trigger.
>
>There is one important limitation to this approach: every callback function triggered by the state transition must be able to handle all of the arguments. This may cause problems if the callbacks each expect somewhat different data.
>
>To get around this, Transitions supports an alternate method for sending data. If you set `send_event=True` at Machine initialization, all arguments to the triggers will be wrapped in an EventData instance and passed on to every callback. (The EventData object also maintains internal references to the source state, model, transition, machine, and trigger associated with the event, in case you need to access these for anything.)

### Beispiel aus Doku: Very asynchronous Dancing: 
https://github.com/pytransitions/transitions/blob/master/examples/Playground.ipynb

In [ ]:
from transitions.extensions.asyncio import AsyncMachine
import asyncio

class Dancer:
    
    states = ['start', 'left_food_left', 'left', 'right_food_right']
    
    def __init__(self, name, beat):
        self.my_name = name
        self.my_beat = beat
        self.moves_done = 0
        
    async def on_enter_start(self):
        self.moves_done += 1
        
    async def wait(self):
        print(f'{self.my_name} stepped {self.state}')
        await asyncio.sleep(self.my_beat)

    async def dance(self):
        while self.moves_done < 5:
            await self.step() #Acthung, dieser State muss mit 
        
dancer1 = Dancer('Tick', 0.5)
dancer2 = Dancer('Tock', 0.55)

m = AsyncMachine(model=[dancer1, dancer2], states=Dancer.states, initial='start', after_state_change='wait')
m.add_ordered_transitions(trigger='step')

# it starts okay but becomes quite a mess
_ = await asyncio.gather(dancer1.dance(), dancer2.dance()) 

## Benuzte AsyncState explizit
Ziel ist die Fähigkeit das State Design Pattern nach GoF zu programmieren

Beachte Antwort von Alexander Neumann ([aleneum](https://github.com/aleneum)) auf  [SO](https://stackoverflow.com/questions/78853312/how-to-use-asyncstate-class-of-pytransitions-asyncio-extension-explicitly/78855445#78855445 )

In [ ]:
from transitions.extensions.asyncio import AsyncMachine, AsyncState, AsyncEventData
import asyncio
import nest_asyncio
nest_asyncio.apply()


class StartState(AsyncState):
    def __init__(self, name, *args, **kwargs) -> None:
        super().__init__(name, *args, **kwargs)
        self.add_callback('enter', self.on_enter_state)

    async def on_enter_state(self, eventdata: AsyncEventData):
        print("On_enter start state")
        eventdata.model.moves_done += 1


class Dancer:
    states = [StartState(name='start'), 'left_food_left', 'left', 'right_food_right']

    def __init__(self, name, beat):
        self.my_name = name
        self.my_beat = beat
        self.moves_done = 0

    async def wait(self, event_data: AsyncEventData):
        print(f'{self.my_name} stepped {self.state}')
        await asyncio.sleep(self.my_beat)

    async def dance(self):
        while self.moves_done < 5:
            await self.step()


dancer1 = Dancer('Tick', 1)
dancer2 = Dancer('Tock', 1.1)

m = AsyncMachine(model=[dancer1, dancer2], states=Dancer.states, initial='start', after_state_change='wait', send_event=True)
m.add_ordered_transitions(trigger='step')

async def main():
    await asyncio.gather(dancer1.dance(), dancer2.dance())

asyncio.run(main())

Tick stepped left_food_left
Tock stepped left_food_left
Tick stepped left
Tock stepped left
Tick stepped right_food_right
Tock stepped right_food_right
On_enter start state
Tick stepped start
On_enter start state
Tock stepped start
Tick stepped left_food_left
Tock stepped left_food_left
Tick stepped left
Tock stepped left
Tick stepped right_food_right
Tock stepped right_food_right
On_enter start state
Tick stepped start
On_enter start state
Tock stepped start
Tick stepped left_food_left
Tock stepped left_food_left
Tick stepped left
Tock stepped left
Tick stepped right_food_right
Tock stepped right_food_right
On_enter start state
Tick stepped start
Tick stepped left_food_left
On_enter start state
Tock stepped start
Tick stepped left
Tock stepped left_food_left
Tick stepped right_food_right
Tock stepped left
On_enter start state
Tick stepped start
Tock stepped right_food_right
Tick stepped left_food_left
On_enter start state
Tock stepped start
Tick stepped left
Tock stepped left_food_lef

In [1]:
from transitions import Machine

class Watch_PV_SM:
    '''
    Watch_PV_SM Klasse
    ==================

    Das ist der Observer für Laser ProzessVariablen und TriggerID.
    Die TriggerID wird von Raspberry PI erzeugt, nachdem es Trigger Signale am GPIO registriert. Dadurch entsteht 
    dem Triggersignal und Publizieren der TriggerId ein Delay.
    Weil es möglich ist, dass schnelle Geräte ihre Werte kurz vor der TriggerID publizieren und langsame Geräte 
    dagegen danach, wird Watch_PV_SM als StateMachine realisiert:

    States
    -------
    - Grundzustand: watch_PV
    - waiting_for_PV_update
    - waiting_for_trigger
    - write_to_db
    - timeout
    - error

    Szenarien
    ----------

    1. Szenario 1
        - Wenn neue TriggerID registriert wird, erfolgt ein Wechsel in Zustand: waiting_for_PV_update
        - waiting_for_PV_update wartet auf Änderung der PV. Passiert diese Änderung werden die Daten in DB geschrieben
        - write_to_db und Übergang in Grundzustand

    2. Szenario 2    
        - Wenn zuerst ein PV Update erfolgt, wechselt der Automat in Zustand: waiting_for_trigger
        - u.s.w  
    '''


    states=['watch_PV', 'waiting_for_trigger','waiting_for_PV_update', 'write_to_db', 'timeout', 'error']

    def __init__(self) -> None:
        self.SM = Machine(model=Watch_PV_SM, states=Watch_PV_SM.states)

In [8]:
from transitions.extensions.asyncio import AsyncMachine
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [3]:
class SimpleMachine(object):

    states = ['dummy', 'start', 'waiting']

    def __init__(self, stop_event):
        self.machine = AsyncMachine(model=self, states=SimpleMachine.states, initial='dummy')
        self.machine.add_transition('run', 'dummy', 'start')
        self.machine.add_transition('timeoutTransition', '*', 'waiting')
        self.machine.add_transition('stop_waiting', 'waiting', 'start')
        self.stop_event = stop_event


    async def doing_things(self):
        try:
            await asyncio.wait_for(self.stop_event.wait(), timeout=10.0)
            print("Event was set!")
        except asyncio.TimeoutError:
            print("Timeout reached, event was not set .")
        
        # print("work work")
        # await asyncio.sleep(10)

    async def on_enter_start(self):
        try:
            await asyncio.wait_for(self.doing_things(), 5)
        except asyncio.TimeoutError:
            print("Timeout!")
            await self.timeoutTransition()

    async def on_enter_waiting(self):
        print("Enter wait state")
        
test_machine = None
test_SM = None

# def main():
#     # logging.basicConfig(level=logging.DEBUG)
#     global test_machine
#     global test_SM
#     test_machine = SimpleMachine()
#     print("State Machine started")
#     asyncio.get_event_loop().run_until_complete(test_machine.run())

async def main():
    global test_machine
    global test_SM
    test_machine = SimpleMachine()
    print("State Machine started")
    test_SM = asyncio.create_task(test_machine.run())

asyncio.run(main())

State Machine started


work work
Timeout!
Enter wait state


In [9]:
from __future__ import annotations  # Ermöglicht Vorwärtsdefinitionen 
from transitions.extensions.asyncio import AsyncMachine, AsyncState
from datetime import datetime, time
import asyncio

# Definition von asynchronen Zuständen
class IdleState(AsyncState):
    async def on_enter(self, event_data):
        heater = event_data.model
        print("Entering idle state. Heizung aus.")
        await self.do(heater)
    
    async def on_exit(self, event_data):
        print("Exiting idle state. Heizung an.")
    
    async def do(self, heater):
        print("Idle...")
        #while True: 
        try:
            await asyncio.wait_for(heater.switch_event.wait(), timeout=1.0)
            print("Manual Event was set! Go to Heating State")
            heater.switch_event.clear()
            heater.heating_on()
           # break
        except asyncio.TimeoutError:
            if heater.current_temperature > heater.raumtemp:
                heater.current_temperature -= 0.5
                print(f"Cooling down... Current temperature: {heater.current_temperature}°C")
            for vremja in heater.on_times:
                switchtime = datetime.strptime(vremja, "%H:%M").time()
                now = datetime.now().time()
                if now > switchtime:
                    await heater.heating_on()
                  #  break


class HeatingState(AsyncState):       
    async def on_enter(self, event_data):
        heater = event_data.model
        print("Entering heating state. Heizung an.")
        await self.do(heater)
    
    async def on_exit(self, event_data):
        print("Exiting heating state. Heizung aus.")
    
    async def do(self, heater):
        #while True: 
        try:
            await asyncio.wait_for(heater.switch_event.wait(), timeout=1.0)
            print("Manual Event was set! Go to Idle State")
            heater.switch_event.clear()
            heater.heating_off()
            #break
        except asyncio.TimeoutError:
            if heater.current_temperature < heater.targettemp:
                heater.current_temperature += 0.5
                print(f"Heating... Current temperature: {heater.current_temperature}°C")
            for vremja in heater.of_times:
                switchtime = datetime.strptime(vremja, "%H:%M").time()
                now = datetime.now().time()
                if now > switchtime:
                    await heater.heating_off()
                #    break

# Definition der Heizungssteuerung
class Heater:
    def __init__(self, switchevent: asyncio.Event, raumtemp = 20, targettemp = 30, on_times=["05:00", "16:00"], of_times=["06:30","19:00"]):
        self.states = {
            'idle': IdleState(name='idle'),
            'heating': HeatingState(name='heating')
        }
        self.machine = AsyncMachine(model=self, states=list(self.states.values()), initial='idle')
        self.current_temperature = 18  # Beispielstarttemperatur
        self.raumtemp = raumtemp
        self.targettemp = targettemp
        self.on_times = on_times
        self.of_times = of_times
        self.switch_event = switchevent
    
    async def run(self):
        print("Running Heater...")
        while True:
            await self.machine.state().do(self)

# Maschine mit Zuständen initialisieren
switch_event = asyncio.Event()
heater = Heater(switch_event, raumtemp=20, targettemp=30)

# Übergänge hinzufügen
heater.machine.add_transition('heating_on', 'idle', 'heating')
heater.machine.add_transition('heating_off', 'heating', 'idle')

async def main():
    heater_task = asyncio.create_task(heater.run())
    await asyncio.sleep(10)
    switch_event.set()  # Manuelles Event setzen, um Zustand zu wechseln
    await asyncio.sleep(5)
    heater_task.cancel()

# Asynchronen Code ausführen
asyncio.run(main())


Running Heater...


In [55]:
await test_machine.timeoutTransition()


DEBUG:transitions.extensions.asyncio:Executed machine preparation callbacks before conditions.
DEBUG:transitions.extensions.asyncio:Initiating transition from state dummy to state waiting...
DEBUG:transitions.extensions.asyncio:Executed callbacks before conditions.
DEBUG:transitions.extensions.asyncio:Executed callback before transition.
DEBUG:transitions.extensions.asyncio:Exiting state dummy. Processing callbacks...
INFO:transitions.extensions.asyncio:Finished processing state dummy exit callbacks.
DEBUG:transitions.extensions.asyncio:Entering state waiting. Processing callbacks...


wait for input


INFO:transitions.extensions.asyncio:Finished processing state waiting enter callbacks.
DEBUG:transitions.extensions.asyncio:Executed callback after transition.
DEBUG:transitions.extensions.asyncio:Executed machine finalize callbacks


finisch


True